In [1]:
from typing import List
import util
import torch
import numpy as np
import os
import pandas as pd


df = pd.read_csv(util.WBCATT_TRAIN_CSV)
df['label'].unique()

/home/e19372/anaconda3/envs/biomedclip/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/e19372/anaconda3/envs/biomedclip/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/e19372/anaconda3/envs/biomedclip/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


array(['Neutrophil', 'Eosinophil', 'Basophil', 'Lymphocyte', 'Monocyte'],
      dtype=object)

In [2]:
df.head(20)

,img_name,label,cell_size,cell_shape,nucleus_shape,nuclear_cytoplasmic_ratio,chromatin_density,cytoplasm_vacuole,cytoplasm_texture,cytoplasm_colour,granule_type,granule_colour,granularity,path
0,BNE_7323.jpg,Neutrophil,big,round,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_7323.jpg
1,BNE_7938.jpg,Neutrophil,big,irregular,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_7938.jpg
2,BNE_8741.jpg,Neutrophil,big,irregular,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_8741.jpg
3,BNE_9071.jpg,Neutrophil,big,irregular,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_9071.jpg
4,BNE_9373.jpg,Neutrophil,small,round,unsegmented-round,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_9373.jpg
5,BNE_9763.jpg,Neutrophil,big,irregular,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_9763.jpg
6,BNE_12387.jpg,Neutrophil,small,round,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_12387.jpg
7,BNE_12405.jpg,Neutrophil,big,round,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_12405.jpg
8,BNE_14932.jpg,Neutrophil,small,round,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_14932.jpg
9,BNE_16131.jpg,Neutrophil,big,irregular,unsegmented-band,low,densely,no,clear,light blue,small,pink,yes,PBC_dataset_normal_DIB/neutrophil/BNE_16131.jpg


In [3]:
# 1. load model, process, and tokenizer
model, preprocess, tokenizer = util.load_clip_model()
print("Model, preprocess, and tokenizer loaded successfully.")

# 2. load dataset - MODIFIED FOR CHEXPERT
features, labels = util.extract_embeddings(
    model=model,
    preprocess=preprocess,
    split="test",
)

# Convert to tensors - MODIFIED FOR MULTI-OBSERVATION SUPPORT
all_feats = torch.from_numpy(features).float()
all_labels = torch.from_numpy(labels).long()

print(f"Loaded {len(all_feats)} wbcatt embeddings")

blood_types = ["basophil", "eosinophil",
               "lymphocyte", "monocyte", "neutrophil"]
template = "A photo of a {blood_type}"
prompt_set = tuple(template.format(blood_type=bt) for bt in blood_types)

results = util.evaluate_prompt_set(
    prompt_set, all_feats, all_labels, model, tokenizer)
print("Baseline Evaluation results:\n", results)

/home/e19372/anaconda3/envs/biomedclip/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/e19372/anaconda3/envs/biomedclip/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Model, preprocess, and tokenizer loaded successfully.
Creating WBCAtt dataset and dataloader for test split...
Label to index mapping (hardcoded): {'Basophil': 0, 'Eosinophil': 1, 'Lymphocyte': 2, 'Monocyte': 3, 'Neutrophil': 4}
Index to label mapping: {0: 'Basophil', 1: 'Eosinophil', 2: 'Lymphocyte', 3: 'Monocyte', 4: 'Neutrophil'}
Labels converted to indices: [1, 0, 3, 4, 4, 4, 1, 1, 0, 1, 3, 3, 3, 4, 4, 2, 0, 3, 2, 4, 0, 4, 1, 0, 3, 3, 3, 4, 0, 3, 3, 3, 3, 4, 4, 4, 4, 2, 1, 4, 4, 3, 3, 0, 4, 4, 1, 3, 4, 4, 1, 1, 4, 4, 4, 4, 2, 3, 0, 0, 1, 2, 1, 3, 3, 0, 1, 4, 0, 1, 0, 0, 3, 4, 0, 2, 1, 4, 4, 0, 4, 4, 3, 4, 4, 2, 1, 0, 4, 0, 3, 4, 4, 0, 1, 0, 3, 4, 1, 1, 0, 1, 4, 4, 1, 0, 4, 4, 4, 3, 0, 1, 1, 2, 2, 4, 1, 2, 4, 1, 3, 1, 2, 1, 3, 1, 3, 0, 4, 3, 4, 3, 3, 4, 2, 0, 4, 1, 4, 1, 3, 1, 4, 4, 4, 1, 2, 4, 1, 0, 2, 4, 3, 4, 1, 0, 1, 1, 1, 3, 4, 4, 2, 4, 4, 4, 4, 1, 1, 2, 1, 1, 4, 2, 2, 1, 4, 2, 1, 4, 1, 4, 1, 0, 4, 2, 1, 4, 1, 4, 0, 2, 2, 4, 3, 4, 1, 2, 2, 1, 1, 4, 0, 4, 1, 4, 4, 4, 4, 1, 2, 1,

Extracting WBCAtt test embeddings: 100%|██████████| 97/97 [00:10<00:00,  8.83it/s]


Caching embeddings to ./wbcatt_cache/wbcatt_test_features.npy and ./wbcatt_cache/wbcatt_test_labels.npy...
Extracted embeddings shape: (3099, 512)
Labels shape: (3099,)
Unique labels: [0 1 2 3 4]
Loaded 3099 wbcatt embeddings
Baseline Evaluation results:
 {'accuracy': 0.23814133591481124, 'cm': array([[  0,   0,   2,  68, 279],
       [  0,   0,   0,  51, 876],
       [  0,   0,   0, 228, 139],
       [  0,   0,   0,  83, 362],
       [  0,   0,   0, 356, 655]]), 'report': '              precision    recall  f1-score   support\n\n           0     0.0000    0.0000    0.0000       349\n           1     0.0000    0.0000    0.0000       927\n           2     0.0000    0.0000    0.0000       367\n           3     0.1056    0.1865    0.1348       445\n           4     0.2834    0.6479    0.3943      1011\n\n    accuracy                         0.2381      3099\n   macro avg     0.0778    0.1669    0.1058      3099\nweighted avg     0.1076    0.2381    0.1480      3099\n', 'inverted_ce': 0.00

In [11]:
CLASSES = sorted(['Neutrophil', 'Eosinophil', 'Basophil', 'Lymphocyte', 'Monocyte'])
template = "This blood film provides a distinct visualization of <cell_type>"
optimized_prompt_set =  ("Displays a nucleus obscured by a profusion of dark purple granules, releasing histamine and heparin; granules often show metachromatic shifts with Wright's stain; cell diameter 8-15 μm.", "Features large, intensely eosinophilic granules filled with major basic protein and eosinophil cationic protein, appearing as bright orange-red inclusions with Wright's stain; cell size 12-17 μm.", "Demonstrates a large, round nucleus with a smooth chromatin pattern and a narrow rim of pale blue cytoplasm, often exhibiting a slight perinuclear halo with Wright's stain; cell size 6-8 μm.", "Presents a kidney-shaped nucleus with a delicate chromatin network and abundant, grey-blue cytoplasm containing numerous vacuoles, giving a foamy appearance with Wright's stain; cell size 14-18 μm.", "Exhibits a segmented nucleus with 3-5 lobes connected by thin chromatin strands, and numerous fine, lilac-colored granules containing myeloperoxidase and defensins, visible with Wright's stain; cell size 10-12 μm.")  
optimized_results = util.evaluate_prompt_set(
    optimized_prompt_set, all_feats, all_labels, model, tokenizer)
print("Optimized Evaluation results:\n", optimized_results)

Optimized Evaluation results:
 {'accuracy': 0.5262988060664731, 'cm': array([[  7,  39,  48,   0, 255],
       [  0, 610, 139,   0, 178],
       [  1,  17, 294,   0,  55],
       [  5,  18, 113,   0, 309],
       [  1,  57, 233,   0, 720]]), 'report': '              precision    recall  f1-score   support\n\n           0     0.5000    0.0201    0.0386       349\n           1     0.8232    0.6580    0.7314       927\n           2     0.3555    0.8011    0.4925       367\n           3     0.0000    0.0000    0.0000       445\n           4     0.4746    0.7122    0.5696      1011\n\n    accuracy                         0.5263      3099\n   macro avg     0.4307    0.4383    0.3664      3099\nweighted avg     0.4995    0.5263    0.4673      3099\n', 'inverted_ce': 0.011222573098874438, 'inverted_weighted_ce': 0.00800696797787039, 'f1_macro': 0.36641298518816184, 'f1_weighted': 0.4672802136564399}
